In [1]:
import numpy as np
import pandas as pd

# from sklearn.metrics import mean_squared_error, confusion_matrix, accuracy_score
from sklearn.preprocessing import  StandardScaler
# from sklearn.decomposition import PCA

# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold

# import keras
# from keras import models, layers, callbacks, constraints, backend, activations
from keras.models import model_from_json
from sklearn.utils import resample


# from matplotlib import pyplot
# import matplotlib.patches as mpatches

# import datetime
import pathlib
import time
# import json
# import sys
import os

# %matplotlib inline
# pd.options.display.max_columns = None

from Utility import scale, ann_model, string_time

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## load datasets

In [2]:
train_dataset = pd.DataFrame.from_csv('../dataset/train.csv', index_col=None)
test_dataset = pd.DataFrame.from_csv('../dataset/test.csv', index_col=None)

/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  if __name__ == '__main__':
/home/luis/anaconda3/envs/tf-source-keras/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  from ipykernel import kernelapp as app


## split training into X_train, y_train & rename train_dataset into X_test

In [3]:
y_train = pd.get_dummies(train_dataset['Cover_Type'],  prefix='Cover_Type_')
y_single = train_dataset['Cover_Type']
X_train = train_dataset.drop('Cover_Type', axis=1)

X_train.shape, y_train.shape, y_single.shape

((15120, 55), (15120, 7), (15120,))

In [4]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train))

### keras ann bagging

### ann params

In [5]:
features = X_train.shape[1]
output_size = y_train.shape[1]
batch_size = 1

hidden_layers = [
    {'neurons': 100, 'dropout': 0.1 },
    {'neurons': 100, 'dropout': 0.1 },
    {'neurons': 100, 'dropout': 0.1 },
    {'neurons': 100, 'dropout': 0.1 },
]

# activation = ['relu', 'softmax'],
# optimizer = 'adam',
# loss = 'categorical_crossentropy',

noise = (True, 0.01)

# metrics = ['accuracy']

### training params

In [6]:
# number of classifiers
base_clf_count = 10
sample_size = 0.6

epochs = 100
verbose = 0

filepath = '../models'

bootstarp_size = int(sample_size * X_train.shape[0])

### next model suffix

In [ ]:
model_number = len(os.listdir(filepath))
if model_number % 2 == 1:
    model_number = model_number - 1
model_number = int(model_number / 2)
model_number

1

### generate models for bootstrapping

In [ ]:
start = time.time()

for i in range(base_clf_count): # loop for bagging

    X_sample = resample(X_train_scaled, n_samples = bootstarp_size, replace=True) # sampling features
    
    y_sample = y_train.loc[X_sample.index] # get sample of labels with same index as sample of features
    
    model = ann_model(
        features = features,
        output_size = output_size,
        batch_size = batch_size,
        hidden_layers = hidden_layers,
        noise = (True, 0.01),
    )
    
    model.fit(X_sample, y_sample, verbose = verbose, epochs = epochs)

    pathlib.Path(filepath).mkdir(parents=True, exist_ok=True)
    
    with open('{}/model_{}.json'.format(filepath, model_number), 'w') as json_file:
        json_file.write(model.to_json()) # serialize model to JSON
        json_file.close()
    
    model.save_weights('{}/model_{}.hdf5'.format(filepath, model_number))

    model_number = model_number + 1

end = time.time()


print('time to train {} models : {}'.format(base_clf_count, string_time(start, end)))